# 如何从消息对象中解析文本

:::info 前置条件

本指南假设您已熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models/)
- [消息](/docs/concepts/messages/)
- [输出解析器](/docs/concepts/output_parsers/)
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel/)

:::

LangChain [消息](/docs/concepts/messages/) 对象支持多种格式的内容，包括文本、[多模态数据](/docs/concepts/multimodality/)和[内容块](/docs/concepts/messages/#aimessage)字典列表。

[聊天模型](/docs/concepts/chat_models/) 响应内容的格式可能取决于提供商。例如，[Anthropic](/docs/integrations/chat/anthropic/) 的聊天模型对于典型的字符串输入将返回字符串内容：

In [ ]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-haiku-latest")

response = llm.invoke("你好")
response.content

'Hi there! How are you doing today? Is there anything I can help you with?'

但是，当生成工具调用时，响应内容被结构化为传达模型推理过程的内容块：

In [ ]:
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """获取某地的天气。"""

    return "晴天。"


llm_with_tools = llm.bind_tools([get_weather])

response = llm_with_tools.invoke("旧金山，加州的天气怎么样？")
response.content

[{'text': "I'll help you get the current weather for San Francisco, California. Let me check that for you right away.",
  'type': 'text'},
 {'id': 'toolu_015PwwcKxWYctKfY3pruHFyy',
  'input': {'location': 'San Francisco, CA'},
  'name': 'get_weather',
  'type': 'tool_use'}]

为了自动解析消息对象中的文本，而不管底层内容的格式如何，我们可以使用 [StrOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)。我们可以将其与聊天模型组合如下：

In [3]:
from langchain_core.output_parsers import StrOutputParser

chain = llm_with_tools | StrOutputParser()

[StrOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) 简化了从消息对象中提取文本的过程：

In [ ]:
response = chain.invoke("旧金山，加州的天气怎么样？")
print(response)

I'll help you check the weather in San Francisco, CA right away.


这在流式上下文中特别有用：

In [ ]:
for chunk in chain.stream("旧金山，加州的天气怎么样？"):
    print(chunk, end="|")

|I'll| help| you get| the current| weather for| San Francisco, California|. Let| me retrieve| that| information for you.||||||||||

有关更多信息，请参阅 [API 参考](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)。